# Final Project

- Saige Belanger
    - (20951877)
- Dylan Faelker
    - (20960747)
- Ethan Liu
    - (20959615)
- Timothy Zheng
    - t54zheng (20939203)

In [1]:
import pandas as pd
from pandasql import sqldf
import matplotlib.pyplot as plt
import numpy as np
import warnings
from sklearn import linear_model
import statsmodels.api as sm
import scipy.stats as stats
from math import sqrt

from sklearn import preprocessing
from sklearn.impute import SimpleImputer
import datetime as dt

warnings.filterwarnings('ignore')

In [2]:
all_monthly_data = pd.read_sas("merged_df.sas7bdat", encoding = 'ISO-8859-1')

In [3]:
all_monthly_data.drop(["ticker", "conm", "gvkey", "cusip", "naics", "gsubind"], axis=1, inplace=True) # We don't use these columns anyway, drop them

In [4]:
factors = ['IM', 'range_20', 'log_vol_dollar_20',
       'range_120', 'log_vol_dollar_120', 'xret_5', 'xret_10', 'xret_20',
       'xret_indsize_20', 'xret_indsize_std20', 'xret_40', 'xret_120',
       'xret_indsize_120', 'xret_indsize_std120', 'KDJ_20', 'deviation_pct20',
       'MoneyFlowIndex_20', 'RSI_20', 'KDJ_120', 'deviation_pct120',
       'MoneyFlowIndex_120', 'RSI_120', 'IV_capm', 'mdr', 'ami_3', 'beta_3y',
       'beta_5y', 'tail_2y', 'dp', 'leverage', 'BL', 'roe', 'roa',
       'profitability', 'sales_g_q', 'sales_g_ttm', 'op_income_g_q', 'ni_g_q',
       'op_income_g_ttm', 'ni_g_ttm', 'sue_NI', 'BM', 'AM', 'EP', 'SP',
       'roe_q', 'roa_q', 'Cto', 'pe_ttm', 'lag_log_size']

ret_cols = ['ret_f1', 'ret_f2', 'ret_f3', 'ret_f4', 'ret_f5', 'ret_f6', 
            'ret_f7', 'ret_f8', 'ret_f9', 'ret_f10', 'ret_f11', 'ret_f12']

In [5]:
non_data_cols = [x for x in all_monthly_data.columns if x not in factors and x not in ret_cols]
non_data_cols

['permno', 'yyyymm', 'monthid', 'PRC', 'VOL', 'SHROUT']

In [6]:
# Inputation - as in ML Lecture 1

# Drop NA in all non-numerical columns
all_monthly_data.dropna(subset=non_data_cols, inplace=True)

grouped_med = all_monthly_data.groupby(by='monthid')
# the lambda function gets the median per group in the groupby object, and fills the NaN values with the median per group
imputed_grouped = grouped_med.transform(lambda y: y.fillna(y.median()))

# This line assigns the values of the medians 
all_monthly_data = all_monthly_data.assign(**imputed_grouped.to_dict(orient='series'))
all_monthly_data.dropna(inplace=True)

In [ ]:
# Filtering data by min price and min market share for each year

all_monthly_data['yyyy'] = all_monthly_data['yyyymm'].astype(str).str[:4]
all_monthly_data['MKTSHR'] = all_monthly_data['PRC'] * all_monthly_data['SHROUT'] * 1_000

to_drop_indices = []

for permno in all_monthly_data.permno.unique():
    for year in all_monthly_data['yyyy'].unique():
        mask = (all_monthly_data['permno'] == permno) & (all_monthly_data['yyyy'] == year)
        if all_monthly_data[mask].shape[0] != 0 != 0 and (all_monthly_data[mask]['MKTSHR'].iloc[0] < 100_000_000 or all_monthly_data[mask]['PRC'].iloc[0] <= 5):
            to_drop_indices += list(all_monthly_data[mask].index)
all_monthly_data.drop(to_drop_indices, inplace=True)

In [7]:
# Winsorizing factors--should winsorize the variables by quarter
for column in factors:
    for date in set(list(all_monthly_data["monthid"])):
        mask = (all_monthly_data["monthid"] == date)
        
        std = all_monthly_data[column][mask].std()
        mean = all_monthly_data[column][mask].mean()

        upper = mean + 3 * std
        lower = mean - 3 * std
        
        all_monthly_data[column][mask].clip(lower, upper, inplace= True)

In [8]:
all_monthly_data

,permno,yyyymm,monthid,IM,range_20,log_vol_dollar_20,range_120,log_vol_dollar_120,xret_5,xret_10,...,ret_f3,ret_f4,ret_f5,ret_f6,ret_f7,ret_f8,ret_f9,ret_f10,ret_f11,ret_f12
1,10026.0,198602.0,74.0,0.106458,0.021404,13.486234,0.019091,13.232742,0.000561,0.007616,...,0.406593,-0.156250,-0.375000,-0.066667,-0.166667,0.114286,0.051282,-0.048780,0.615385,0.031746
2,10026.0,198603.0,75.0,-0.183465,0.021967,13.935166,0.020239,13.429522,0.001357,0.002957,...,-0.156250,-0.375000,-0.066667,-0.166667,0.114286,0.051282,-0.048780,0.615385,0.031746,0.030769
3,10026.0,198604.0,76.0,0.636488,0.023080,13.849557,0.020850,13.536907,-0.005400,0.000191,...,-0.375000,-0.066667,-0.166667,0.114286,0.051282,-0.048780,0.615385,0.031746,0.030769,-0.119403
4,10026.0,198605.0,77.0,0.354652,0.023095,13.798978,0.022117,13.612165,0.007211,0.006643,...,-0.066667,-0.166667,0.114286,0.051282,-0.048780,0.615385,0.031746,0.030769,-0.119403,-0.042373
5,10026.0,198606.0,78.0,0.308972,0.020076,13.643324,0.022047,13.697346,-0.009126,-0.005370,...,-0.166667,0.114286,0.051282,-0.048780,0.615385,0.031746,0.030769,-0.119403,-0.042373,0.159292
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
440715,93429.0,201908.0,476.0,0.043659,0.022145,17.094448,0.022864,17.153811,0.007966,0.013102,...,0.035693,0.009251,0.026833,-0.071904,-0.217105,0.113501,0.074864,-0.123802,-0.059820,0.051425
440716,93429.0,201909.0,477.0,0.017751,0.025800,17.123301,0.023830,17.149012,-0.000186,-0.005715,...,0.009251,0.026833,-0.071904,-0.217105,0.113501,0.074864,-0.123802,-0.059820,0.051425,-0.044122
440717,93429.0,201910.0,478.0,0.004530,0.024727,17.193758,0.024310,17.139314,0.004466,-0.006312,...,0.026833,-0.071904,-0.217105,0.113501,0.074864,-0.123802,-0.059820,0.051425,-0.044122,-0.073513
440718,93429.0,201911.0,479.0,0.005354,0.022660,17.061345,0.024140,17.111865,-0.004312,0.000399,...,-0.071904,-0.217105,0.113501,0.074864,-0.123802,-0.059820,0.051425,-0.044122,-0.073513,0.128552


## Factor Code

In [10]:
ff4_factors = pd.read_sas("ff4_factors.sas7bdat", encoding = 'ISO-8859-1')
ff4_factors["monthid"] = ff4_factors.index + 1
ff4_factors.head()

,DATEFF,SMB,HML,MKTRF,RF,UMD,monthid
0,1980-01-31,0.0162,0.0175,0.0551,0.0080,0.0755,1
1,1980-02-29,-0.0185,0.0061,-0.0122,0.0089,0.0788,2
2,1980-03-31,-0.0664,-0.0101,-0.1290,0.0121,-0.0955,3
3,1980-04-30,0.0105,0.0106,0.0397,0.0126,-0.0043,4
4,1980-05-30,0.0213,0.0038,0.0526,0.0081,-0.0112,5


In [11]:
dates = [int(x) for x in sorted(list(set(list(all_monthly_data["yyyymm"]))))]
dates[0], dates[-1]

(198004, 201912)

In [12]:
monthids = [int(x) for x in sorted(list(set(list(all_monthly_data["monthid"]))))]
monthids[0], monthids[-1], len(monthids)

(4, 480, 477)

In [13]:
testing_range = monthids[0:2*(len(monthids) // 3)]
validation_range = monthids[2 * len(monthids) // 3:]

# Validate that ranges have correct ratios
len(testing_range) / len(monthids), len(validation_range) / len(monthids), len(testing_range) + len(validation_range)

(0.6666666666666666, 0.3333333333333333, 477)

## Testing Factors

In [14]:
model_factors = ['IM', 'range_20', 'log_vol_dollar_20',
       'range_120', 'log_vol_dollar_120', 'xret_5', 'xret_10', 'xret_20',
       'xret_indsize_20', 'xret_indsize_std20', 'xret_40', 'xret_120',
       'xret_indsize_120', 'xret_indsize_std120', 'KDJ_20', 'deviation_pct20',
       'MoneyFlowIndex_20', 'RSI_20', 'KDJ_120', 'deviation_pct120',
       'MoneyFlowIndex_120', 'RSI_120', 'IV_capm', 'mdr', 'ami_3', 'beta_3y',
       'beta_5y', 'tail_2y', 'dp', 'leverage', 'BL', 'roe', 'roa',
       'profitability', 'sales_g_q', 'sales_g_ttm', 'op_income_g_q', 'ni_g_q',
       'op_income_g_ttm', 'ni_g_ttm', 'sue_NI', 'BM', 'AM', 'EP', 'SP',
       'roe_q', 'roa_q', 'Cto', 'pe_ttm', 'lag_log_size']

In [30]:
all_monthly_data = pd.merge(ff4_factors, all_monthly_data, on="monthid")

## [m, n, l] model for Fama-MacBeth Double Regression
We will use the technique employed during Assignment 2, utilizing a 36-month lookback for factor data to generate our betas (**First Stage**)
* For period $t_i$, we will use data starting at $t_{i-36} ... t_{i-1}$ if available. Worst case we look for 12 prior samples.

In [40]:
permnos = set(all_monthly_data["permno"])

In [ ]:
# Fama-Macbeth
# using [m,n,l] model 

summary_results["monthid"] = []
summary_results["permno"] = []
for factor in model_factors:
    summary_results[f"{factor}_beta"] = []
    summary_results[f"{factor}_beta_t_stat"] = []

for permno in permnos:
    for (i, monthid) in enumerate(testing_range):
        window = set(testing_range[max(0, i-35):i+1]) # t_(i-36) to t_(i-1) returns. Compare to t_i returns
        window_data = all_monthly_data[all_monthly_data["monthid"].isin(window) & all_monthly_data["permno"] == permno]
        
        if len(window_data) < 12:
            continue
            
        window_factors = window_data[model_factors]
        window_returns = window_data["RET"] # Since factors are from t-1
        rf = window_data["RF"]
        
        window_excess_returns = window_returns - rf
    
        model_x = sm.add_constant(window_factors)
        model_y = sm.OLS(window_excess_returns, model_x).fit()
    
        summary_results["monthid"].append(monthid)
        summary_results["permno"].append(monthid)
    
        for factor in model_factors:
            summary_results[f"{factor}_beta"].append(model_y.params[factor])
            summary_results[f"{factor}_beta_t_stat"].append(model_y.pvalues[factor])

In [39]:
first_stage_df = pd.DataFrame(summary_results).set_index("monthid")
first_stage_df

,IM_beta,range_20_beta,log_vol_dollar_20_beta,range_120_beta,log_vol_dollar_120_beta,xret_5_beta,xret_10_beta,xret_20_beta,xret_indsize_20_beta,xret_indsize_std20_beta,...,sue_NI_beta_t_stat,BM_beta_t_stat,AM_beta_t_stat,EP_beta_t_stat,SP_beta_t_stat,roe_q_beta_t_stat,roa_q_beta_t_stat,Cto_beta_t_stat,pe_ttm_beta_t_stat,lag_log_size_beta_t_stat
monthid,,,,,,,,,,,,,,,,,,,,,
15,-0.084999,0.738747,0.005370,-1.515312,-0.004024,0.122831,-0.000789,0.088851,-0.048722,-2.355734,...,0.172842,0.773014,0.224980,0.340740,0.001050,0.058404,0.237818,0.005511,0.002195,0.021564
16,-0.083052,0.818305,0.007308,-1.501797,-0.005971,0.087309,0.033256,0.125887,-0.056150,-2.576230,...,0.242161,0.676430,0.364770,0.770359,0.002518,0.125214,0.304488,0.013880,0.003211,0.008000
17,-0.079448,0.806973,0.006679,-1.356769,-0.005976,0.058366,0.025935,0.130709,-0.077833,-2.318278,...,0.256043,0.529827,0.284685,0.629816,0.001543,0.075577,0.524272,0.006951,0.022082,0.005211
18,-0.077680,0.945136,0.007259,-1.341518,-0.007539,0.067895,0.019369,0.154335,-0.107188,-2.416389,...,0.434524,0.118413,0.139002,0.948848,0.002009,0.052418,0.628701,0.004683,0.099531,0.006240
19,-0.050301,1.179976,0.005949,-1.248748,-0.007391,0.031233,0.036791,0.227151,-0.202883,-2.466497,...,0.225213,0.034573,0.087653,0.190669,0.006746,0.049453,0.737247,0.012067,0.575799,0.029156
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
317,-0.035109,0.328822,0.001923,0.181577,-0.004345,-0.050486,0.084731,-0.013465,0.072009,0.131035,...,0.392799,0.012359,0.181641,0.548547,0.000169,0.385583,0.405169,0.446920,0.001818,0.534794
318,-0.036375,0.283434,0.000450,0.284175,-0.003220,-0.059491,0.081863,-0.019580,0.081020,0.137067,...,0.324449,0.014411,0.008985,0.012918,0.000009,0.832680,0.280971,0.773557,0.010746,0.209043
319,-0.017243,-0.245234,0.000083,0.638405,-0.003052,-0.063294,0.078475,0.001695,0.055151,0.254017,...,0.313938,0.101871,0.094184,0.038758,0.000051,0.677560,0.077073,0.400110,0.005578,0.145375


# Performance Analysis

In [72]:
def total_ret(port_ret):
    return port_ret.sum()
    # return np.prod(port_ret + 1) - 1

def tracking_error(port_ret, bench_ret):
    return (port_ret - bench_ret).std()

def information_ratio(port_ret, bench_ret):
    return (total_ret(port_ret) - total_ret(bench_ret)) / tracking_error(port_ret, bench_ret)

def sharpe_ratio(port_ret, rf_ret):
    return information_ratio(port_ret, rf_ret)

def sharpe_ratio(port_xret):
    return total_ret(port_xret) / port_xret.std()